# 752. Open the Lock

## Topic Alignment
- Exploring combination locks via BFS mimics search over discrete configuration spaces, similar to hyperparameter enumeration with forbidden states.


## Metadata 摘要
- Source: https://leetcode.com/problems/open-the-lock/
- Tags: BFS, String, State Graph
- Difficulty: Medium
- Priority: High

## Problem Statement 原题描述
You have a lock represented by a 4-digit string. You can turn each wheel forward or backward by one (with wrap-around). Given a set of deadends and a target combination, find the minimum number of turns to open the lock from "0000", or return -1 if impossible.

## Progressive Hints
- Hint 1: Each lock state is a four-digit string; neighbors are reached by incrementing or decrementing one digit mod 10.
- Hint 2: Avoid deadend states by storing them in a `blocked` set.
- Hint 3: BFS layers correspond to rotations performed; stop when the target state is dequeued.


## Solution Overview
Use BFS starting from `'0000'` provided it is not blocked. For each state, generate up to eight neighbors by adjusting each digit ±1 modulo 10. Enqueue unseen states that are not in the deadend set. The level at which you discover the target yields the minimal number of turns; if BFS exhausts without reaching it, return `-1`.


## Detailed Explanation
1. Convert `deadends` into a set for `O(1)` membership checks. If `'0000'` is blocked, return `-1`.
2. Initialize a queue with `'0000'` and a visited set containing it.
3. Set `depth = 0`. While the queue has nodes:
   - For all states in the current level, pop a state `code`.
   - If `code == target`, return `depth`.
   - For each of four positions, rotate forward and backward modulo 10 to form neighbors.
   - Enqueue any neighbor absent from `blocked` and `visited`, marking it visited immediately.
   - After processing the level, increment `depth`.
4. If the queue empties, return `-1`.


## Complexity Trade-off Table
| Approach | Time | Space | Notes |
| --- | --- | --- | --- |
| BFS over lock states | O(10⁴) | O(10⁴) | Explores each state at most once |
| Bidirectional BFS | O(10⁴ / 2) | O(10⁴) | Faster in practice by meeting in the middle |
| Dijkstra | O(10⁴ log 10⁴) | O(10⁴) | Equivalent here; all edges have weight 1 |


In [ ]:
from collections import deque
from typing import List

class Solution:
    def openLock(self, deadends: List[str], target: str) -> int:
        dead = set(deadends)
        start = "0000"
        if start in dead:
            return -1
        if target == start:
            return 0
        queue = deque([(start, 0)])
        visited = {start}
        while queue:
            state, depth = queue.popleft()
            for i in range(4):
                digit = int(state[i])
                for move in (-1, 1):
                    nd = (digit + move) % 10
                    neighbor = state[:i] + str(nd) + state[i+1:]
                    if neighbor in dead or neighbor in visited:
                        continue
                    if neighbor == target:
                        return depth + 1
                    visited.add(neighbor)
                    queue.append((neighbor, depth + 1))
        return -1


In [ ]:
tests = [
    ((['0201','0101','0102','1212','2002'], '0202'), 6),
    ((['8888'], '0009'), 1),
    ((['8887','8889','8878','8898','8788','8988','7888','9888'], '8888'), -1)
]
solver = Solution()
for (dead, target), expected in tests:
    assert solver.openLock(dead, target) == expected
print('All tests passed.')


## Complexity Analysis
- Time: O(10⁴) in the worst case because there are only 10⁴ possible states.
- Space: O(10⁴) for the queue and visited set.


## Edge Cases & Pitfalls
- Check whether the target also lies in the deadend set; return `-1` immediately.
- Mark states as visited when enqueuing, not when dequeuing, to prevent duplicates.
- Handle wraparound carefully: incrementing `9` should produce `0` and decrementing `0` should produce `9`.


## Follow-up Variants
- Return the actual sequence of rotations by recording parent states.
- Introduce per-wheel costs and switch to Dijkstra or A* to account for differing rotation weights.
- Expand to locks with more wheels or different alphabets by adjusting the neighbor generation logic.


## Takeaways
- BFS on compact state spaces is easy to implement and guarantees optimality for uniform edge weights.
- Pre-eliminating deadends prevents futile branches and keeps the queue small.
- Modulo arithmetic simplifies wheel rotations, avoiding long chains of `if` statements.


## Similar Problems
| Problem ID | Problem Title | Technique |
| --- | --- | --- |
| LC 127 | Word Ladder | BFS on word states |
| LC 433 | Minimum Genetic Mutation | BFS on 8-letter strings |
| LC 909 | Snakes and Ladders | BFS on board positions |
